In [0]:
from mlflow.tracking import MlflowClient
import mlflow

# Set the registry URI for Unity Catalog
mlflow.tracking._model_registry.utils._get_registry_uri_from_spark_session = lambda: "databricks-uc"

# Initialize the MLflow client
client = MlflowClient()

# Search for all registered models
registered_models = client.search_registered_models()

# Iterate over models in the specified namespace and delete their versions and registrations
for model in registered_models:
    if model.name.startswith("field_demos.ml_ops"):
        print(f"Processing model: {model.name}")
        # Retrieve all versions of this model
        model_versions = client.search_model_versions(f"name='{model.name}'")
        if not model_versions:
            print("  No versions found.")
        else:
            for version in model_versions:
                print(f"  Deleting version: {version.version} (Run ID: {version.run_id})")
                client.delete_model_version(model.name, version.version)
        # Delete the registered model
        print(f"  Deleting registered model: {model.name}")
        client.delete_registered_model(model.name)
        print("-" * 50)